한경 컨센서스의 report가 pdf형태로 존재. 

pdf_directly_to_text에서는 pdfminer를 활용하여 pdf다운로드 없이 url을 이용하여 \
바로 text 형태로 출력하려는 시도.

2가지의 문제점 발생
- 1. title은 다르지만 내용은 동일한 report 다수 존재
- 2. 중복report제거를 위해 여러가지 시도 중 메모리 부족 등으로 인한 connection broken error 발생

In [1]:
import pandas as pd
import numpy as np

In [3]:
from bs4 import BeautifulSoup
import requests

### Crawling


In [4]:
# 크롤링할 페이지 url
# 10년치 기업 종목(2011-11-04 ~ 2021-11-03) 1996 페이지 약 16만개 pdf파일
# list목록 수를 80개 설정 후 크롤링

url_list = []
for page in range(1, 2): # test로 1page 80개 목록만
    url = 'http://consensus.hankyung.com/apps.analysis/analysis.list?&sdate=2011-11-04&edate=2021-11-03\
    &report_type=CO&pagenum=80&order_type=&now_page={}'.format(page)
        # header 정보를 넣어줘야 크롤링 가능
    headers = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
        'referer': 'http://consensus.hankyung.com/apps.analysis/analysis.list?&skinType=business'   
    }
    res = requests.get(url, headers=headers)
    html = res.content
    soup = BeautifulSoup(html, 'html.parser')
    f_url = 'http://consensus.hankyung.com'
    soup.select('#contents > div.table_style01 > table > tbody > tr > td')
    pdf_list=soup.select('#contents > div > table > tbody > tr > td.text_l > a')
    for i in range(len(pdf_list)):
        b_url = soup.select('#contents > div > table > tbody > tr > td.text_l > a')[i]['href']
        title = soup.select('#contents > div > table > tbody > tr > td.text_l > a')[i].text
        full_url = f_url+b_url
        print(full_url, title)
        url_list.append(full_url)

    pages = page
    print()
    print()
    print('----------------------------', pages,'page', len(pdf_list) ,'----------------------------')

http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600357 카카오뱅크(323410)포트폴리오 급변 시기에는 순마진율을 봐야
http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600354 삼성SDI(006400)EV 향 원형전지가 효자
http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600353 랩지노믹스(084650)[3Q21 Review 2020 년 연간 실적을 넘어서는 ...
http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600352 카카오뱅크(323410)펀더멘탈은 변함이 없다
http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600351 삼성SDI(006400)실적 성장세는 계속된다
http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600341 효성중공업(298040)기대되는 4분기
http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600339 카카오뱅크(323410)순조롭게 항해 중
http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600338 삼성전기(009150)계절성보다 변화에 주목
http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600337 삼성SDI(006400)EV향 이익 확대
http://consensus.hankyung.com/ap

http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600190 아프리카TV(067160)리레이팅에는 ‘쉬는 시간’이 없다
http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600189 엘앤에프(066970)2차전지 소재기업 최선호주
http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600182 SK이노베이션(096770)Green Identity 구축은 현재 진행 형
http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600180 LG화학(051910)LG화학 NDR 후기
http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600179 호텔신라(008770)[ 3 Q2 1 Review] 과열된 시장


---------------------------- 1 page 80 ----------------------------


In [10]:
url_list[1]

'http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600354'

In [5]:
len(url_list)

80

## pdf directly to txt

In [6]:
import io
import requests
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

In [7]:
def pdf_from_url_to_txt(url):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
    'referer': 'http://consensus.hankyung.com/apps.analysis/analysis.list?&skinType=business'}
    
    f= requests.get(url, headers=headers)
    
    #  to Bytes
    fp = io.BytesIO(f.content)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    for page in PDFPage.get_pages(fp,
                                  pagenos,
                                  maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue() #  to string : retstr = StringIO()
    retstr.close()
    return str.replace('\n', '')

In [95]:
import os
import time
start = time.time()

if not os.path.isdir('./pdf_to_text/'):
    os.mkdir('./pdf_to_text/')
text_path = './pdf_to_text/'
for li in url_list:
    with open( text_path + li.split('=')[1]  +'.txt', 'w', encoding='utf8') as f:
        f.write(pdf_from_url_to_txt(li))

print("time :", time.time() - start)

time : 661.0441339015961


### report 제목은 다르지만 내용은 같은 report 존재

In [11]:
pdf_from_url_to_txt('http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600293').

'               \x0c 미디어/엔터      2 │ IBKS RESEARCH \x0c이환욱  6915-5473     CONTENTS [산업분석] 분야, 대륙, 주요 국가(미국, 일본) 동향 가장 높은 성장률을 기록한 ‘대한민국’ ‘보는(시각적)음악’ 전성시대 차별화된 성공 포인트 SNS/칼군무 글로벌 인재 양성소 ‘K-엔터테인먼트’ 독보적인 기획력과 체계적인 시스템 글로벌 뮤직 인더스트리 2020 ................................................................................................... 10 K-Pop은 어떻게 Global 음악 시장 중심에 섰나? ................................................................ 18 좋은 재료와 생산설비는 K-POP의 지속 가능성을 높인다 ................................................. 23 K-엔터테인먼트사, ‘테크’ 기업으로 진화 중 ......................................................................... 28 비즈니스 모델의 혁신, ‘팬 플랫폼’ 디지털 자산(feat NFT)과 메타버스 확장의 주인공 [기업분석] 하이브 (352820)............................................................................................................................. 36 혁신에 혁신을 거듭하다 / 매수(신규) / 목표주가: 500,000원 에스엠 (041510).............................................................................................................................

In [12]:
pdf_from_url_to_txt('http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600292')

'               \x0c 미디어/엔터      2 │ IBKS RESEARCH \x0c이환욱  6915-5473     CONTENTS [산업분석] 분야, 대륙, 주요 국가(미국, 일본) 동향 가장 높은 성장률을 기록한 ‘대한민국’ ‘보는(시각적)음악’ 전성시대 차별화된 성공 포인트 SNS/칼군무 글로벌 인재 양성소 ‘K-엔터테인먼트’ 독보적인 기획력과 체계적인 시스템 글로벌 뮤직 인더스트리 2020 ................................................................................................... 10 K-Pop은 어떻게 Global 음악 시장 중심에 섰나? ................................................................ 18 좋은 재료와 생산설비는 K-POP의 지속 가능성을 높인다 ................................................. 23 K-엔터테인먼트사, ‘테크’ 기업으로 진화 중 ......................................................................... 28 비즈니스 모델의 혁신, ‘팬 플랫폼’ 디지털 자산(feat NFT)과 메타버스 확장의 주인공 [기업분석] 하이브 (352820)............................................................................................................................. 36 혁신에 혁신을 거듭하다 / 매수(신규) / 목표주가: 500,000원 에스엠 (041510).............................................................................................................................

### 중복값 제거 ver_1

In [30]:
import os
import time
start = time.time()

if not os.path.isdir('./pdf_to_text/'):
    os.mkdir('./pdf_to_text/')
text_path = './pdf_to_text/'


#  중복값이 연달아 나올 것이라는 가정하에 바로전 text와 다음 text의  글자 100개의 일치여부 활용
for i in range(len(url_list)):
    if i> 1:
        if pdf_from_url_to_txt(url_list[i])[0:100] == pdf_from_url_to_txt(url_list[i-1])[0:100] :
            pass

        else :

            with open( text_path + url_list[i].split('=')[1]  +'.txt', 'w', encoding='utf8') as f:
                f.write(pdf_from_url_to_txt(url_list[i]))

print("time :", time.time() - start)

# 결과적으로 중복값이 연달아 존재 하지 않는 경우가 많음
# 또한 서버 불안정과 if문 추가로 인한 메모리 부족 이유로  ChunkedEncodingError 발생

ChunkedEncodingError: ('Connection broken: IncompleteRead(7401 bytes read, 695 more expected)', IncompleteRead(7401 bytes read, 695 more expected))

### Connection broken 해결

In [33]:
def pdf_from_url_to_txt(url):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
    'referer': 'http://consensus.hankyung.com/apps.analysis/analysis.list?&skinType=business'}
    
    r = ''
    while r == '':
        try:
            f= requests.get(url, headers=headers)
        except (requests.exceptions.ConnectionError, requests.exceptions.ChunkedEncodingError) as err:
            print(err + ' Put to sleep before retrying.')
            time.sleep(100)
            continue
            
    
    # Cast to StringIO object
        fp = io.BytesIO(f.content)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos = set()
        for page in PDFPage.get_pages(fp,
                                      pagenos,
                                      maxpages=maxpages,
                                      password=password,
                                      caching=caching,
                                      check_extractable=True):
            interpreter.process_page(page)
        fp.close()
        device.close()
        str = retstr.getvalue()
        retstr.close()
        return str.replace('\n', '')

In [ ]:
# import os
# import time
# start = time.time()

# if not os.path.isdir('./pdf_to_text/'):
#     os.mkdir('./pdf_to_text/')
# text_path = './pdf_to_text/'

# for i in range(len(url_list)):
#     if i> 1:
#         if pdf_from_url_to_txt(url_list[i])[0:100] == pdf_from_url_to_txt(url_list[i-1])[0:100] :
#             pass

#         else :

#             with open( text_path + url_list[i].split('=')[1]  +'.txt', 'w', encoding='utf8') as f:
#                 f.write(pdf_from_url_to_txt(url_list[i]))

# print("time :", time.time() - start)

### 중복값 제거 ver_2(size활용)
- 보통 중복값은 여러 종목을 하나의 리포트에 모아둔 점 확인
- 따라서 50KB이상의 text는 text 변환 미실시 적용

In [95]:
url = 'http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=600293'
headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
    'referer': 'http://consensus.hankyung.com/apps.analysis/analysis.list?&skinType=business'}
f= requests.get(url, headers=headers)
fp = io.BytesIO(f.content)
# seek과 tell을 사용하여 size 출력
fp.seek(0,2)
size = fp.tell()
size

4481144

In [93]:
def pdf_from_url_to_txt(url):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
    'referer': 'http://consensus.hankyung.com/apps.analysis/analysis.list?&skinType=business'}
    
    f= requests.get(url, headers=headers)
    
    
    fp = io.BytesIO(f.content)
    fp.seek(0,2)
    size = fp.tell()
    
    if size < 2000000 :  # 2000000 = 약 50kB
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos = set()
        for page in PDFPage.get_pages(fp,
                                      pagenos,
                                      maxpages=maxpages,
                                      password=password,
                                      caching=caching,
                                      check_extractable=True):
            interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str.replace('\n', '')

In [94]:
import os
import time
start = time.time()

if not os.path.isdir('./pdf_to_text1/'):
    os.mkdir('./pdf_to_text1/')
text_path = './pdf_to_text1/'
for li in url_list:
    with open( text_path + li.split('=')[1]  +'.txt', 'w', encoding='utf8') as f:
        f.write(pdf_from_url_to_txt(li))

print("time :", time.time() - start)

time : 314.77755975723267


In [ ]:
# 시간이 상당히 오래걸리고 메모리 활용의 비효율성으로 pdf download 후 text로 변환 결정